In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from modules.builder import Builder
import pandas as pd
import os
from modules.cleaner import Cleaner
from modules.qualityChecker import QualityChecker
import json
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import joblib
import plotly.express as px
import numpy as np

ModuleNotFoundError: No module named 'modules'

In [ ]:
data_path = '/usr/src/Covid-Regression-MortalityVSPopulation/datasets/time_series_covid19_confirmed_global.csv'
vac_path = '/usr/src/Covid-Regression-MortalityVSPopulation/datasets/vaccinations.csv'
death_path = '/usr/src/Covid-Regression-MortalityVSPopulation/datasets/all_weekly_excess_deaths.csv'
metadata_path = '/usr/src/Covid-Regression-MortalityVSPopulation/datasets/owid-covid-data.csv'

In [ ]:
df_global = pd.read_csv(data_path)
df_vac = pd.read_csv(vac_path)
df_deaths = pd.read_csv(death_path)
df_meta = pd.read_csv(metadata_path)

In [ ]:
df_global

In [ ]:
df_global.loc[df_global['Country/Region'] == 'Thailand']

In [ ]:
df_global = df_global.drop(["Province/State", 'Lat', 'Long'], axis=1)

In [ ]:
df_global = df_global.rename({"Country/Region": 'country'}, axis='columns')

In [ ]:
df = df_global.loc[df_global['country'] == 'Thailand']

In [ ]:
def reverse_columns(df : pd.DataFrame):
    new_df = pd.DataFrame()
    country_list = list()
    dates = list()
    data = list()
    for j in df.columns[1:]:
        country_list.append("Thailand")
        dates.append(j)
        data.append(df[j].tolist()[0])
    new_df['country'] = country_list
    new_df['date'] = dates
    new_df['effected_patients'] = data
    return new_df

In [ ]:
df = reverse_columns(df)

In [ ]:
df

In [ ]:
df_deaths.loc[df_deaths['country'] == 'Thailand']

In [ ]:
df_meta.loc[df_meta['location'] == 'Thailand']

In [ ]:
df = df_meta.loc[df_meta['location'] == 'Thailand']

In [ ]:
df.reset_index()

In [ ]:
df.info()

In [ ]:
df.duplicated().sum()

In [ ]:
df.isna().sum()

In [ ]:
df = df.fillna(0)

In [ ]:
df.isna().sum()

In [ ]:
df['total_deaths']

In [ ]:
df.info()

In [ ]:
df = df.drop(['iso_code', 'continent', 'location'], axis=1)

In [ ]:
df

In [ ]:
df['date'] = pd.to_datetime(df['date'])    
df['date_delta'] = (df['date'] - df['date'].min())  / np.timedelta64(1,'D')

In [ ]:
df

In [ ]:
df.columns

In [ ]:
df = df.drop("date", axis=1)
train_data = df.copy()
train_data = train_data.drop(['new_deaths', 'tests_units'], axis=1)
x = train_data[['total_cases', 'new_cases', 'new_cases_smoothed', 'total_deaths', 'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'total_tests', 'new_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'total_boosters',
       'new_vaccinations', 'new_vaccinations_smoothed',
       'total_vaccinations_per_hundred', 'people_vaccinated_per_hundred',
       'people_fully_vaccinated_per_hundred', 'total_boosters_per_hundred',
       'new_vaccinations_smoothed_per_million',
       'new_people_vaccinated_smoothed',
       'new_people_vaccinated_smoothed_per_hundred', 'stringency_index',
       'population', 'population_density', 'median_age', 'aged_65_older',
       'aged_70_older', 'gdp_per_capita', 'extreme_poverty',
       'cardiovasc_death_rate', 'diabetes_prevalence', 'female_smokers',
       'male_smokers', 'handwashing_facilities', 'hospital_beds_per_thousand',
       'life_expectancy', 'human_development_index',
       'excess_mortality_cumulative_absolute', 'excess_mortality_cumulative',
       'excess_mortality', 'excess_mortality_cumulative_per_million',
       'date_delta']]
y = df[['new_deaths']]


In [ ]:
x.reset_index()
y.reset_index()

In [ ]:
y

In [ ]:
model = Builder().build_model("LinearRegression")

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=100)

In [ ]:
# Standard Normalization(x)
std_x = StandardScaler()
x_train = std_x.fit_transform(x_train)
x_test = std_x.transform(x_test)

# Standard Normalization(y)
std_y = StandardScaler()
y_train = std_y.fit_transform(y_train)
y_test = std_y.transform(y_test)

In [ ]:
model.fit(x_train, y_train)

In [ ]:
y_pred = std_y.inverse_transform(model.predict(x_test))

In [ ]:
y_pred